In [1]:
import pandas as pd
from pinecone import Pinecone
from pinecone import ServerlessSpec
import os
import time
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
import json
import pandas as pd
from langdetect import detect
from googletrans import Translator

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\aleta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
df = pd.read_parquet('./acl-publication-info.74k.parquet')

In [2]:
# Create a serverless index
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "academic-papers"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name, 
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    )

while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)
pinecone_index = pc.Index(index_name)

In [5]:
ids = []
for id in pinecone_index.list():
    ids.extend(id)
    print(id)

['1952.earlymt-1.1', '1952.earlymt-1.13', '1952.earlymt-1.17', '1952.earlymt-1.2', '1952.earlymt-1.26', '1952.earlymt-1.7', '1956.earlymt-1.3', '1957.earlymt-1.12', '1960.earlymt-fsmtw.2', '1960.earlymt-fsmtw.4', '1960.earlymt-fsmtw.5', '1960.earlymt-fsmtw.6', '1960.earlymt-nsmt.13', '1960.earlymt-nsmt.31', '1960.earlymt-nsmt.32', '1960.earlymt-nsmt.4', '1960.earlymt-nsmt.67', '1961.earlymt-1.10', '1961.earlymt-1.14', '1961.earlymt-1.23', '1961.earlymt-1.3', '1961.earlymt-1.30', '1961.earlymt-1.4', '1961.earlymt-1.6', '1962.earlymt-1.1', '1962.earlymt-1.4', '1962.earlymt-1.5', '1962.earlymt-1.6', '1971.earlymt-1.1', '1971.earlymt-1.11', '1971.earlymt-1.12', '1971.earlymt-1.14', '1971.earlymt-1.3', '1971.earlymt-1.4', '1971.earlymt-1.7', '1971.earlymt-1.8', '1976.earlymt-1.12', '1976.earlymt-1.13', '1976.earlymt-1.2', '1976.earlymt-1.20', '1976.earlymt-1.23', '1976.earlymt-1.25', '1976.earlymt-1.3', '1976.earlymt-1.6', '1976.earlymt-1.8', '1978.tc-1.3', '1978.tc-1.4', '1978.tc-1.5', '19

In [9]:
i = 0
for id in ids:
    i+=1
    url = df.loc[df['acl_id'] == id]['url'].values[0]
    response = pinecone_index.update(id=id, set_metadata={'url':url})
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
